# Sentiment Analysis with Attention Mechanism
- author: Eu-Bin KIM
- date: 15th of September 2021


In [1]:
"""
목표  = 감성분석기, LSTM.
"""
from typing import List, Tuple
import numpy as np
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from torch.nn import functional as F
import torch

## TODO - 1 어텐션 마스크 만들기
- 어텐션 마스크란?
- 왜 마스크를 만들어야 할까?
- `torch.where` 

In [2]:
DATA: List[Tuple[str, int]] = [
    # 긍정적인 문장 - 1
    ("나는 자연어처리가 좋아.", 1),
    ("나는 자연어처리.", 1),
    ("도움이 되었으면.", 1),
    # 병국님
    ("오늘도 수고했어.", 1),
    # 영성님
    ("너는 할 수 있어.", 1),
    # 정무님
    ("오늘 내 주식이 올랐다.", 1),
    # 우철님
    ("오늘 날씨가 좋다.", 1),
    # 유빈님
    ("난 너를 좋아해.", 1),
    # 다운님
    ("지금 정말 잘하고 있어.", 1),
    # 민종님
    ("지금처럼만 하면 잘될거야.", 1),
    ("사랑해.", 1),
    ("저희 허락없이 아프지 마세요.", 1),
    ("오늘 점심 맛있다.", 1),
    ("오늘 너무 예쁘다.", 1),
    # 다운님
    ("곧 주말이야.", 1),
    # 재용님
    ("오늘 주식이 올랐어.", 1),
    # 병운님
    ("우리에게 빛나는 미래가 있어.", 1),
    # 재용님
    ("너는 참 잘생겼어.", 1),
    # 윤서님
    ("콩나물 무침은 맛있어.", 1),
    # 정원님
    ("강사님 보고 싶어요.", 1),
    # 정원님
    ("오늘 참 멋있었어.", 1),
    # 예은님
    ("맛있는게 먹고싶다.", 1),
    # 민성님
    ("로또 당첨됐어.", 1),
    # 민성님
    ("이 음식은 맛이 없을수가 없어.", 1),
    # 경서님
    ("오늘도 좋은 하루보내요.", 1),
    # 성민님
    ("내일 시험 안 본대.", 1),
    # --- 부정적인 문장 - 레이블 = 0
    ("난 너를 싫어해.", 0),
    # 병국님
    ("넌 잘하는게 뭐냐?.", 0),
    # 선희님
    ("너 때문에 다 망쳤어.", 0),
    # 정무님
    ("오늘 피곤하다.", 0),
    # 유빈님
    ("난 삼성을 싫어해.", 0),
    ("진짜 가지가지 한다.", 0),
    ("꺼져.", 0),
    ("그렇게 살아서 뭘하겠니.", 0),
    # 재용님 - 주식이 파란불이다?
    ("오늘 주식이 파란불이야.", 0),
    # 지현님
    ("나 오늘 예민해.", 0),
    ("주식이 떨어졌다.", 0),
    ("콩나물 다시는 안먹어.", 0),
    ("코인 시즌 끝났다.", 0),
    ("배고파 죽을 것 같아.", 0),
    ("한강 몇도냐.", 0),
    ("집가고 싶다.", 0),
    ("나 보기가 역겨워.", 0),  # 긍정적인 확률이 0
    # 진환님
    ("잘도 그러겠다.", 0),
    ("너는 어렵다.", 0),
    # ("자연어처리는", 0)
]

In [3]:
    # 문장을 다 가져온다
    sents = [sent for sent, label in DATA]
    # 레이블
    labels = [label for sent, label in DATA]
    # 정수인코딩
    tokenizer = Tokenizer(char_level=True, filters=" ")
    tokenizer.fit_on_texts(texts=sents)

In [5]:
# builders # - 기존의 build_X로는 부족하다
def build_X(sents: List[str], tokenizer: Tokenizer, max_length: int) -> Tuple[torch.Tensor, torch.Tensor]:
    seqs = tokenizer.texts_to_sequences(texts=sents)
    seqs = pad_sequences(sequences=seqs, padding="post", maxlen=max_length, value=0)
    X = torch.LongTensor(seqs)  # (N, L)
    return X

In [6]:
build_X(sents, tokenizer, max_length=30)

tensor([[10,  8,  1,  ...,  0,  0,  0],
        [10,  8,  1,  ...,  0,  0,  0],
        [16, 63,  7,  ...,  0,  0,  0],
        ...,
        [10,  1, 34,  ...,  0,  0,  0],
        [21, 16,  1,  ...,  0,  0,  0],
        [11,  8,  1,  ...,  0,  0,  0]])

In [7]:
# torch.where
x = torch.randn(3, 2)  # 랜덤한 값으로 채워진 벡터
y = torch.ones(3, 2)  # 1로 채워진 벡터
print(x)
print(y)
# x > 0을 만족하는 성분 -> x.
# x > 0을 만족하지않는 성분 -> y로 대체.
torch.where(x > 0, x, y)

tensor([[ 0.6454, -1.3884],
        [ 2.5947,  1.4516],
        [ 1.6640, -0.4085]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


tensor([[0.6454, 1.0000],
        [2.5947, 1.4516],
        [1.6640, 1.0000]])

In [9]:
def build_X_M(sents: List[str], tokenizer: Tokenizer, max_length: int) -> Tuple[torch.Tensor, torch.Tensor]:
    seqs = tokenizer.texts_to_sequences(texts=sents)
    seqs = pad_sequences(sequences=seqs, padding="post", maxlen=max_length, value=0)
    X = torch.LongTensor(seqs)  # (N, L)
    ### TODO 1 ###
    # 마스크 만들기
    # 병국님
    # 마스크. padding 인 경우 = 0, 나머지는 = 1   # (N, L)
    M = torch.where(X > 0, 1, 0)
    # 영성님
    # M = torch.where(X!= 0, 1, 0)
    ##############
    return X, M  # (N, L), (N, L)


def build_y(labels: List[int]) -> torch.Tensor:
    return torch.FloatTensor(labels)

In [ ]:
# d# 영성님: LongTensor는 무슨 역할인가요?
E = torch.nn.Embedding(10, 64)
# 아래 코드는 오류가 뜬다. 
# print(E(torch.Tensor([1, 2, 3])))
# embedding 네트워크가 인덱스의 인자로 LongTensor를 강요하기 때문.
print(E(torch.LongTensor([1, 2, 3])))

tensor([[-0.0960, -0.2445,  0.7203, -1.1089, -0.0039, -0.5724, -0.1358,  1.0667,
         -0.4039, -0.0676,  1.3775,  0.1992,  0.0748, -1.4523,  0.2029,  0.3211,
         -0.2057, -0.2718,  0.6098, -0.0837,  2.1377, -2.1805,  0.1773, -0.7229,
         -0.0042,  1.0837,  0.2496,  1.0362,  0.3276,  0.9227,  1.1572,  0.8317,
         -0.4369, -0.2079,  1.1232,  0.4048, -1.0528, -0.1104, -1.0376,  0.0783,
          0.6627,  0.5550, -0.7398,  2.7190, -1.9709,  0.1465,  1.1108, -0.5183,
          0.7849,  0.8918,  0.3272,  0.3357,  0.0077,  0.4409,  0.0791,  1.2646,
          0.6067,  0.3391, -0.4685,  1.3403,  2.1784, -1.2304, -0.7059, -1.8472],
        [ 1.1365, -0.9120, -0.8792, -0.7431, -1.1540,  0.7606,  0.0700, -0.1169,
         -1.1372,  2.3958, -0.9105,  2.5115, -1.1943, -0.2650,  0.8717,  0.2767,
         -0.5158, -0.2070, -0.2192,  0.2427, -1.7534, -0.8205,  0.2029, -0.6147,
          1.4311, -0.2768,  0.2696, -2.0412,  0.3618, -1.1018,  0.0187, -0.7311,
          0.7045,  0.5839, 

## TODO 2 - `SimpleLSTMWithAttention`의 `forward` 함수 재정의하기

H_last 만을 출력하는게 아니라, H_all 도 출력하는 함수로 변경하기.


In [ ]:
#  이렇게 시작해보세요 ! 미리 바구니 만들어보기.
H_all = torch.zeros(size=(...,...,...))  # (N, L, H)

In [10]:
class SimpleLSTMWithAttention(torch.nn.Module):
    def __init__(self, vocab_size: int, hidden_size: int, embed_size: int):
        super().__init__()
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.E = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_size)
        # 학습해야하는 가중치.
        # 학습해야하는 신경망을 다 정의해보자.
        # 1. 정의를 하는 것 스킵
        # 2. forward에서 데이터의 흐름을 *차원의 변화를 트래킹*하면서 확인
        # 3. (A, B) * (B, C) -> (A, C)
        self.W = torch.nn.Linear(in_features=hidden_size + embed_size, out_features=hidden_size*4)
        self.W_hy = torch.nn.Linear(in_features=hidden_size*2, out_features=1)

    def training_step(self, X: torch.Tensor, M: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        y_pred, _ = self.predict(X, M)  # (N, L) -> (N, 1)
        y_pred = torch.reshape(y_pred, y.shape)  # y와 차원의 크기를 동기화
        loss = F.binary_cross_entropy(y_pred, y)  # 이진분류 로스
        loss = loss.sum()  # 배치 속 모든 데이터 샘플에 대한 로스를 하나로
        return loss

    def forward(self, X: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        h_t = f_W(h_t-1, xt)
        :param X: (N, L) = (배치의 크기, 문장의 길이)
        :return: (H_all, H_last) (N, L, H), (N, H)
        """
        # 이제 뭐해요?
        # 반복문
        # 0 -> ???=L-1
        C_t = torch.zeros(size=(X.shape[0], self.hidden_size))  # (?=N, ?=H)
        H_t = torch.zeros(size=(X.shape[0], self.hidden_size))  # (?=N, ?=H)
        H_all = torch.zeros(size=(X.shape[0],X.shape[1],self.hidden_size))   # (N, L, H)
        ### TODO 2 ###  
        # H_all (문장에 대한 모든 단기기억 만들기.)
        for time in range(X.shape[1]):
            # 여기선 뭐하죠?
            X_t = X[:, time]  # (N, L) -> (N, 1)
            X_t = self.E(X_t)  # (N, 1) -> (N, E)
            # 이제 뭐하죠?
            H_cat_X = torch.cat([H_t, X_t], dim=1)  # (N, H), (N, E) -> (?=N,?=H+E)
            G = self.W(H_cat_X)  # (N, H+E) * (H+E, H*4) -> (N, H*4)
            F = G[:, :self.hidden_size]  # 큰 박스 안에 작은 박스를 넣는다.
            I = G[:, self.hidden_size:self.hidden_size*2]  # 큰 박스 안에 작은 박스를 넣는다.
            O = G[:, self.hidden_size*2:self.hidden_size*3]  # 큰 박스 안에 작은 박스를 넣는다.
            H_temp = G[:, self.hidden_size*3:self.hidden_size*4]  # 큰 박스 안에 작은 박스를 넣는다.
            # 이제 뭐하죠? - 활성화 함수,
            F = torch.sigmoid(F)  #  (N ,H) -> (N, H)
            I = torch.sigmoid(I)  #  (N ,H) -> (N, H)
            O = torch.sigmoid(O)  #  (N ,H) -> (N, H)
            H_temp = torch.tanh(H_temp)  #  (N ,H) -> (N, H)
            # 이제 뭐하죠?
            # 1. F의 용도? = 지우개
            # 2. I의 용도? = 필요한 기억 저장
            # 3. O의 용도? = 장기기억으로 부터 최종 단기기억 생성
            C_t = torch.mul(F, C_t) + torch.mul(I, H_temp)
            H_t = torch.mul(O, torch.tanh(C_t))
            H_all[:, time] = H_t  # 모든 N개의 데이터, 이 시간대 = H_t
        ##############
        H_last = H_t
        return H_all, H_last  # (N, L, H), (N, H).


## TODO 3 - `SimpleLSTMWithAttention`의 `predict`함수 재정의 하기

- `torch.einsum` 적극 활용하기!
  - 공식문서: https://pytorch.org/docs/stable/generated/torch.einsum.html
  - Einsum is all you need: https://rockt.github.io/2018/04/30/einsum
  - 간략한 정리(한글): https://ita9naiwa.github.io/numeric%20calculation/2018/11/10/Einsum.html 


In [16]:
# einsum = Domain Specific Language
a = torch.Tensor([1, 2, 3])  #(i, )
b = torch.Tensor([4, 5, 6])  #(i, )
# 이둘을 성분곱을 하고 싶다?
print(torch.mul(a, b))  # 파이토치를 알아야 한다.
print(torch.einsum("i,i->i", a, b))  # 파이토치를 알필요가 없다. (성분곱)
print(torch.einsum("i,i->", a, b))  # 파이토치를 알필요가 없다.  (내적) 

tensor([ 4., 10., 18.])
tensor([ 4., 10., 18.])
tensor(32.)


In [20]:
# 그렇다면 행렬의 연산은?
A = torch.randn(size=(3, 10))
B = torch.randn(size=(10, 3))
print(A @ B)  # 행렬 곱
print(torch.einsum("xy,yz->xz", A, B))
# a,b, c, d, ..z  어떤 캐릭터든 가능. 

tensor([[ 2.1256, -0.8011, -3.4900],
        [-0.4371,  1.5851, -3.1955],
        [ 4.8746, -4.2821, -5.1054]])
tensor([[ 2.1256, -0.8011, -3.4900],
        [-0.4371,  1.5851, -3.1955],
        [ 4.8746, -4.2821, -5.1054]])


In [21]:
def predict(self, X: torch.Tensor, M: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
  """
  :param X: (N, L) inputs.
  :param M: (N, L) masks.
  """
  H_all, H_last = self.forward(X)  # (N, L) -> (N, L, H),  (N, H).
  ### TODO 3 - 1 ###
  # 영성님
  S = torch.einsum("nlh,nh->nl", H_all, H_last)
  # 병국님
  S = torch.einsum("nlh,hn->nl", H_all, H_last.T)  # (N, L, H), (N, H) -> (N, L)
  # 어떻게 이해?
  # ein"sum" 
  # sigma_h(A_nlh * B_nh) = -> nl.
  # mask the scores - we must set this to be negative.
  S[M == 0] = float("-inf")
  A = torch.softmax(S, dim=1)  # attention scores.
  # compute weighted average to get the context.
  # 병국님:37
  # C = torch.einsum("NLH, NL -> NH", H_all, S(x),A(o))
  # 천영성10:37
  # C = torch.einsum('nlh,nl -> nh',H_all,H_last(x), A(o))
  C = torch.einsum("nlh,nl->nh", H_all, A) # (N, L, H), (N, L) -> (N, H). (H_all의 가중평균)
  # 정무님.
  # 다만 einsum은 concatenate를 할 때 사용하는 것은 아님
  C_cat_H = torch.cat([C, H_last], dim=1)  # (N, H), (N, H) -> (N, H*2)  (최종 feature)
  ##############
  y_pred = self.W_hy(C_cat_H)  # (N, H*2) * (H*2, 1) -> (N, 1)
  y_pred = torch.sigmoid(y_pred)  # (N, H) -> (N, 1)
  # print(y_pred)
  return y_pred, A

# 함수 등록
SimpleLSTMWithAttention.predict = predict

## Attention Score를 확인해보기!

In [22]:
class Analyser:
    """
    lstm기반 감성분석기.
    """
    def __init__(self, lstm: SimpleLSTMWithAttention, tokenizer: Tokenizer, max_length: int):
        self.lstm = lstm
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, text: str) -> float:
        X, M = build_X_M(sents=[text], tokenizer=self.tokenizer, max_length=self.max_length)
        y_pred, A = self.lstm.predict(X, M)
        attentions = A.squeeze().detach().numpy()
        tokens = [
            "[PAD]" if idx == 0
            else self.tokenizer.index_word[idx]
            for idx in X.detach().squeeze().tolist()
        ]
        data = [
            (token, "{:.2f}".format(att))
            for token, att in zip(tokens, attentions)
        ]
        print(data)
        return y_pred.item()

In [23]:
# train & test
# --- hyper parameters --- #
EPOCHS = 800  # 에폭 
LR = 0.001  # 학습률
HIDDEN_SIZE = 32 
EMBED_SIZE = 12
MAX_LENGTH = 30  # 문장의 최대길이.

# 데이터 셋을 구축. (X=(N, L, 2), y=(N, 1))
X, M = build_X_M(sents, tokenizer, MAX_LENGTH)
y = build_y(labels)
vocab_size = len(tokenizer.word_index.keys())
vocab_size += 1
# lstm with attention으로 감성분석 문제를 풀기.
lstm = SimpleLSTMWithAttention(vocab_size=vocab_size,hidden_size=HIDDEN_SIZE, embed_size=EMBED_SIZE)
optimizer = torch.optim.Adam(params=lstm.parameters(), lr=LR)

for epoch in range(EPOCHS):
    loss = lstm.training_step(X, M, y)
    loss.backward()  # 오차 역전파
    optimizer.step()  # 경사도 하강
    optimizer.zero_grad()  #  다음 에폭에서 기울기가 축적이 되지 않도록 리셋
    print(epoch, "-->", loss.item())

analyser = Analyser(lstm, tokenizer, MAX_LENGTH)
print("##### TRAIN TEST #####")
for sent, label in DATA:
    print(sent, "->", label, analyser(sent))
print("##### TEST #####")
sent = "나는 자연어처리가 좋아"
print(sent, "->", analyser(sent))

0 --> 0.7047582268714905
1 --> 0.7017163038253784
2 --> 0.6989103555679321
3 --> 0.6963252425193787
4 --> 0.6939470767974854
5 --> 0.6917628049850464
6 --> 0.6897616982460022
7 --> 0.6879352927207947
8 --> 0.6862780451774597
9 --> 0.6847861409187317
10 --> 0.6834574937820435
11 --> 0.6822912693023682
12 --> 0.6812868118286133
13 --> 0.6804436445236206
14 --> 0.6797595024108887
15 --> 0.6792297959327698
16 --> 0.6788447499275208
17 --> 0.6785874962806702
18 --> 0.6784316897392273
19 --> 0.6783411502838135
20 --> 0.6782739162445068
21 --> 0.6781893372535706
22 --> 0.6780573725700378
23 --> 0.6778613328933716
24 --> 0.677598774433136
25 --> 0.6772764325141907
26 --> 0.6769067645072937
27 --> 0.6765038967132568
28 --> 0.6760812997817993
29 --> 0.6756494045257568
30 --> 0.6752157807350159
31 --> 0.6747845411300659
32 --> 0.6743565201759338
33 --> 0.6739306449890137
34 --> 0.6735037565231323
35 --> 0.673071563243866
36 --> 0.672629177570343
37 --> 0.6721712350845337
38 --> 0.6716926693916321

In [25]:
# 모델이 긍/부정을 판단할 때, 어떤 글자에 집중을 하는지 확인해보자! 우리의 직관과 일치하는가?